# 提取VGG16卷积层的特征

In [ ]:
# -*- coding: utf-8 -*-
# @Author:Liu Chuanlong
import torch.utils.data as data
import os
from torchvision.models import vgg16
from INSTRE import INSTREclassification as INCLS
import torch
import torch.nn as nn
import numpy as np
from PIL import Image
import cv2
import torchvision.transforms as transforms
import time

class Warp(object):
    def __init__(self, size, interpolation=Image.BILINEAR):
        self.size = int(size)
        self.interpolation = interpolation

    def __call__(self, img):
        return img.resize((self.size, self.size), self.interpolation)

    def __str__(self):
        return self.__class__.__name__ + ' (size={size}, interpolation={interpolation})'.format(size=self.size,interpolation=self.interpolation)
                
def extract_feature(model, inputs):
    model.fc = torch.nn.LeakyReLU(0.1)
    model.eval()

    result = model(inputs)
    result_npy = result.data.numpy()
    
    return result_npy[0]

def get_features(pretrained_model,inputs):
 
    net1 = nn.Sequential(*list(pretrained_model.children())[0])
    out1 = net1(inputs)
    return out1

train_dataset = INCLS('/home/liuchuanloong/amy/deep_retrieval', 'train', transforms.Compose([
                Warp(224),
                # transforms.ToTensor(),
                # transforms.Scale(224),
                transforms.RandomHorizontalFlip(),
                lambda x: torch.from_numpy(np.array(x)).permute(2, 0, 1).float(),
                lambda x: x.index_select(0, torch.LongTensor([2,1,0]))]))

val_dataset = INCLS('/home/liuchuanloong/amy/deep_retrieval', 'test',transforms.Compose([
                Warp(224),
                # transforms.ToTensor(),
                # transforms.Scale(224),
                transforms.RandomHorizontalFlip(),
                lambda x: torch.from_numpy(np.array(x)).permute(2, 0, 1).float(),
                lambda x: x.index_select(0, torch.LongTensor([2,1,0]))]))

# data loader
train_loader = data.DataLoader(train_dataset, batch_size=32, shuffle=True,
                                           num_workers=2)
val_loader = data.DataLoader(val_dataset, batch_size=32, shuffle=False,
                                          num_workers=2)

print('start!!!')
start = time.time()
VGG16 = vgg16(pretrained=True).cuda()
time_elapse = time.time() - start
print('transform to cuda done!!! elapse{}s'.format(time_elapse))

for step, (b_x,b_y) in enumerate(train_loader):  
    inputs = torch.autograd.Variable(b_x[0]).cuda()
    print(get_features(VGG16,inputs))

# resnet18 模型特征提取测试 两种方法

In [ ]:
import torch 
import torch.nn as nn
import torchvision.models as models
from torch.autograd import Variable
 
import time
 
class toyNet(nn.Module):
 
    def __init__(self, pretrained_model, layers):
        super(toyNet, self).__init__()


        self.net1 = nn.Sequential(*list(pretrained_model.children())[:layers[0]])
        self.net2 = nn.Sequential(*list(pretrained_model.children())[layers[0]:layers[1]])
        self.net3 = nn.Sequential(*list(pretrained_model.children())[layers[1]:layers[2]])

 
    def forward(self, x):

        out1 = self.net1(x)
        out2 = self.net2(out1)
        out3 = self.net3(out2)

        return out1, out2, out3

def get_features(pretrained_model, x, layers = [3, 4, 7]):
 
    net1 = nn.Sequential(*list(pretrained_model.children())[:layers[0]])
#     print(net1)
    out1 = net1(x)

    net2 = nn.Sequential(*list(pretrained_model.children())[layers[0]:layers[1]])
#     print(net2)
    out2 = net2(out1)

    net3 = nn.Sequential(*list(pretrained_model.children())[layers[1]:layers[2]])
#     print(net3)
    out3 = net3(out2)

    return out1, out2, out3
 
x = Variable(torch.rand(1,3,224,224))
net = models.resnet18(pretrained=True)
 
if torch.cuda.is_available():
    x = x.cuda()
    net = net.cuda()

start = time.time()
 
o1, o2, o3 = get_features(net, x)
 
print(time.time() - start)
 
print(o1.data.size())
print(o2.data.size())
print(o3.data.size())
 
print('----------------------------------------------')
 
start = time.time()
 
toynet = toyNet(net, [3,4,7])
y1, y2, y3 = toynet(x)
 
print(time.time() - start)
 
print(y1.data.size())
print(y2.data.size())
print(y3.data.size())


# VGG16  提取某一层的特征

In [ ]:
import numpy as np
import torch
from torchvision import models
from torch.autograd import Variable
import torchvision.transforms as transforms
 
class CNNShow():
    def __init__(self, model):
        self.model = model
        self.model.eval()
 
        self.created_image = self.image_for_pytorch(np.uint8(np.random.uniform(150, 180, (224, 224, 3))))
 
 
    def show(self):
        x = self.created_image
        for index, layer in enumerate(self.model):
            print(index,layer)
            x = layer(x)
            print(x)
 
    def image_for_pytorch(self,Data):
        transform = transforms.Compose([
            transforms.ToTensor(),  # range [0, 255] -> [0.0,1.0]
            transforms.Normalize(mean=(0.485, 0.456, 0.406), std=(0.229, 0.224, 0.225))
        ]
        )
        imData = transform(Data)
        imData = Variable(torch.unsqueeze(imData, dim=0), requires_grad=True)
        return imData
if __name__ == '__main__':
 
    pretrained_model = models.vgg16(pretrained=True).features
    CNN = CNNShow(pretrained_model)
    CNN.show()

# VGG16 分类网络

In [ ]:
# -*- coding: utf-8 -*-
# @Author:Liu Chuanlong
import torch.utils.data as data
import os
from torchvision.models import vgg16
from INSTRE import INSTREclassification as  INCLS
import torch
import torch.nn as nn
import numpy as np
from PIL import Image
import cv2
import torchvision.transforms as transforms

class Warp(object):
    def __init__(self, size, interpolation=Image.BILINEAR):
        self.size = int(size)
        self.interpolation = interpolation

    def __call__(self, img):
        return img.resize((self.size, self.size), self.interpolation)

    def __str__(self):
        return self.__class__.__name__ + ' (size={size}, interpolation={interpolation})'.format(size=self.size,interpolation=self.interpolation)

train_dataset = INCLS('/home/liuchuanloong/amy/deep_retrieval', 'train', transforms.Compose([
                Warp(224),
                # transforms.ToTensor(),
                # transforms.Scale(224),
                transforms.RandomHorizontalFlip(),
                lambda x: torch.from_numpy(np.array(x)).permute(2, 0, 1).float(),
                lambda x: x.index_select(0, torch.LongTensor([2,1,0]))]))
val_dataset = INCLS('/home/liuchuanloong/amy/deep_retrieval', 'test',transforms.Compose([
                Warp(224),
                # transforms.ToTensor(),
                # transforms.Scale(224),
                transforms.RandomHorizontalFlip(),
                lambda x: torch.from_numpy(np.array(x)).permute(2, 0, 1).float(),
                lambda x: x.index_select(0, torch.LongTensor([2,1,0]))]))

# train_data = generatedata(train_dataset)

# data loader
train_loader = data.DataLoader(train_dataset, batch_size=32, shuffle=True,
                                           num_workers=2)
val_loader = data.DataLoader(val_dataset, batch_size=32, shuffle=False,
                                          num_workers=2)
print('start load model!!!')
VGG16 = vgg16(pretrained=False)
VGG16.cuda()
print(VGG16)
print('transfered to cuda done!!!')
optimizer = torch.optim.Adam(VGG16.parameters(), lr= 0.1)
loss_func = nn.CrossEntropyLoss()

EPOCH = 1
for epoch in range(EPOCH):
    for step, (b_x,b_y) in enumerate(train_loader):  # 分配 batch data, normalize x when iterate train_loader
        b_y = torch.max(b_y,1)
        print('b_x{}'.format(type(b_x[0])))
        b_y = b_y[1].type(torch.LongTensor)
        print('b_y{}'.format(type(b_y)))
        input = torch.autograd.Variable(b_x[0]).cuda()
        target = torch.autograd.Variable(b_y).cuda()
        output = VGG16(input)  # VGG16 output
        loss = loss_func(output, target).cpu()  # cross entropy loss
        print(loss)
        optimizer.zero_grad()  # clear gradients for this training step
        loss.backward()  # backpropagation, compute gradients
        optimizer.step()  # apply gradients